In [14]:
import pandas as pd
import numpy as np
from keras.layers import Dense, Activation
from keras.models import Sequential

In [3]:
BATCH_SIZE = 32
EPOCHS = 5
TEST_FACTOR = 0.8

In [20]:
text_fields = ('Name', 'Type 1', 'Type 2')
pokemons = pd.read_csv('data/pokemon.csv')
for f in text_fields:
    _, int_col = np.unique(pokemons[f].values, return_inverse=True, axis=0)
    pokemnos[f] = int_col
pokemons.head()

TypeError: unique() got an unexpected keyword argument 'axis'

In [ ]:
combats = pd.read_csv('data/combats.csv', header=None)
combats.columns = ['a', 'b', 'winner']

,#,Name,Type 1,Type 2,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,80,82,83,100,100,80,1,False
3,4,Mega Venusaur,Grass,Poison,80,100,123,122,120,80,1,False
4,5,Charmander,Fire,NaN,39,52,43,60,50,65,1,False


In [6]:
combats.head()

,a,b,winner
0,266,298,298
1,702,701,701
2,191,668,668
3,237,683,683
4,151,231,151


In [7]:
combats.keys()

Index(['a', 'b', 'winner'], dtype='object')

In [8]:
pokemons['Type 1'].head()

0    Grass
1    Grass
2    Grass
3    Grass
4     Fire
Name: Type 1, dtype: object

In [9]:
training = pd.merge(combats, pokemons, left_on='a', right_on='#').merge(pokemons, left_on='b', right_on='#')
training['winner'] = training['winner'] == training['a']
training['winner'] = training['winner'].astype(int)
del training['a']
del training['b']
del training['#_x']
del training['#_y']
training.columns

Index(['winner', 'Name_x', 'Type 1_x', 'Type 2_x', 'HP_x', 'Attack_x',
       'Defense_x', 'Sp. Atk_x', 'Sp. Def_x', 'Speed_x', 'Generation_x',
       'Legendary_x', 'Name_y', 'Type 1_y', 'Type 2_y', 'HP_y', 'Attack_y',
       'Defense_y', 'Sp. Atk_y', 'Sp. Def_y', 'Speed_y', 'Generation_y',
       'Legendary_y'],
      dtype='object')

In [10]:
labels = training['winner']
labels.head()

0    0
1    0
2    0
3    1
4    1
Name: winner, dtype: int64

In [11]:
features = training.shape[1]
size = training.shape[0]
training_size = int(size * TEST_FACTOR * 0.1)

model = Sequential()
model.add(Dense(units=training_size, input_shape=(features,)))
model.add(Activation('relu'))
model.add(Dense(units=training_size))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

test_input = training[:training_size].values
test_output = labels[:training_size].values

model.fit(test_input, test_output)

Epoch 1/10


ValueError: could not convert string to float: 'Flying'

In [ ]:
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

SVG(model_to_dot(model).create(prog='dot', format='svg'))

In [ ]:
validate_input = training[training_size+1:]
validate_output = labels[training_size+1:]

model.evaluate(validate_input, validate_output)